In [2]:
import pandas as pd
from glob import glob

Split to sentences and use regex to extract the value.

In [4]:
import spacy
import re

# Define your fee extraction function
def get_value_from_text(text_content):
    nlp = spacy.blank("cs")  # Replace with `spacy.load("cs")` if Czech model is available.
    nlp.max_length = 1_000_000_000  # Increase the max length limit
    nlp.add_pipe('sentencizer')
    
    doc = nlp(text_content)
    sentences = [sent.text for sent in doc.sents]
    
    # Filter sentences that contain the relevant phrases
    relevant_sentences1 = [sentence for sentence in sentences if "za měsíc" in sentence.lower()]
    relevant_sentences2 = [sentence for sentence in sentences if "měsíčně" in sentence.lower()]
    relevant_sentences3 = [sentence for sentence in sentences if "měsíční školné" in sentence.lower()]
    relevant_sentences4 = [sentence for sentence in sentences if "měsíční poplatek" in sentence.lower()]
    relevant_sentences5 = [sentence for sentence in sentences if "měsíční cena" in sentence.lower()]
    relevant_sentences6 = [sentence for sentence in sentences if "/měsíc" in sentence.lower()]
    relevant_sentences7 = [sentence for sentence in sentences if "úplata za" in sentence.lower()]
    relevant_sentences8 = [sentence for sentence in sentences if "cena školného" in sentence.lower()]


    
    # Combine all relevant sentences
    relevant_sentences = relevant_sentences1 + relevant_sentences2 + relevant_sentences3 + relevant_sentences4 + relevant_sentences5 + relevant_sentences6 + relevant_sentences7 + relevant_sentences8
    
    # Define a regex pattern to extract the fee (number followed by "Kč")
    fee_pattern = r"(?<!\d/)(\b\d{1,3}(?:[.,\s]\d{3})*(?:,-)?)\s*(?:Kč)?\b(?!/\d)"
    
    # Initialize an empty list to store all found fees
    all_fees = []
    
    # Loop through relevant sentences and find all fees
    for sentence in relevant_sentences:
        matches = re.findall(fee_pattern, sentence)
        if matches:
            # Add the found fees to the list (cleaning them by removing spaces)
            all_fees.extend([match.replace(" ", "") for match in matches])
    
    # Return the list of all found fees, or None if no fees were found
    return all_fees if all_fees else None



In [5]:
import spacy
import re

# Define your fee extraction function
def get_sentence_from_text(text_content):
    nlp = spacy.blank("cs")  # Replace with `spacy.load("cs")` if Czech model is available.
    nlp.max_length = 1_000_000_000  # Increase the max length limit
    nlp.add_pipe('sentencizer')
    
    doc = nlp(text_content)
    sentences = [sent.text for sent in doc.sents]
    
    # Filter sentences that contain the relevant phrases
    relevant_sentences1 = [sentence for sentence in sentences if "za měsíc" in sentence.lower()]
    relevant_sentences2 = [sentence for sentence in sentences if "měsíčně" in sentence.lower()]
    relevant_sentences3 = [sentence for sentence in sentences if "měsíční školné" in sentence.lower()]
    relevant_sentences4 = [sentence for sentence in sentences if "měsíční poplatek" in sentence.lower()]
    relevant_sentences5 = [sentence for sentence in sentences if "měsíční cena" in sentence.lower()]
    relevant_sentences6 = [sentence for sentence in sentences if "/měsíc" in sentence.lower()]
    relevant_sentences7 = [sentence for sentence in sentences if "úplata za" in sentence.lower()]
    relevant_sentences8 = [sentence for sentence in sentences if "cena školného" in sentence.lower()]


    
    # Combine all relevant sentences
    relevant_sentences = relevant_sentences1 + relevant_sentences2 + relevant_sentences3 + relevant_sentences4 + relevant_sentences5 + relevant_sentences6 + relevant_sentences7 + relevant_sentences8
    
    
    # Define a regex pattern to extract the fee (number followed by "Kč")
    fee_pattern = r"(?<!\d/)(\b\d{1,3}(?:[.,\s]\d{3})*(?:,-)?)\s*(?:Kč)?\b(?!/\d)"
    
    # Initialize a list to store tuples of (fee, sentence)
    fees_and_sentences = []
    
    # Loop through relevant sentences and find all fees
    for sentence in relevant_sentences:
        matches = re.findall(fee_pattern, sentence)
        if matches:
            # Store the fee along with the sentence it was found in
            for match in matches:
                cleaned_fee = match.replace(" ", "")  # Clean the fee by removing spaces
                fees_and_sentences.append((cleaned_fee, sentence.strip()))  # Store fee and sentence
    
    # Return the list of fees and sentences, or None if no fees were found
    return fees_and_sentences if fees_and_sentences else None




In [4]:
files = glob("/Users/annabrazdova/default/kindergartens/01_text_skolky/*.csv")
files


['/Users/annabrazdova/default/kindergartens/01_text_skolky/extracted_text_0.csv',
 '/Users/annabrazdova/default/kindergartens/01_text_skolky/extracted_text_1.csv',
 '/Users/annabrazdova/default/kindergartens/01_text_skolky/extracted_text_3.csv',
 '/Users/annabrazdova/default/kindergartens/01_text_skolky/extracted_text_2.csv',
 '/Users/annabrazdova/default/kindergartens/01_text_skolky/extracted_text_6.csv',
 '/Users/annabrazdova/default/kindergartens/01_text_skolky/extracted_text_5.csv',
 '/Users/annabrazdova/default/kindergartens/01_text_skolky/extracted_text_4.csv']

In [ ]:
dfs=[]
for file in files:
    df = pd.read_csv(file, index_col=0)
    dfs.append(df)


In [ ]:
for i, df in enumerate(dfs):
    # Apply the function to the 'text' column and create a new column 'fee'
    df['fee'] = df['text'].apply(get_value_from_text)
    
    # Apply the function to the 'text' column and create a new column 'fee_sentence'
    df['sentence'] = df['text'].apply(get_sentence_from_text)
    
    no_fee = df[df['sentence'].isnull()]
    fee = df[~df['sentence'].notnull()]


    no_fee.to_csv(f"/Users/annabrazdova/default/kindergartens/02_no_skolky/no_fee_{i}.csv", index=False)
    fee.to_csv(f"/Users/annabrazdova/default/kindergartens/02_no_skolky/fee_{i}.csv", index=False)
